In [24]:
import numpy as np
import pandas as pd

from typing import List, Tuple
import matplotlib.pyplot as plt
import math

from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from ucimlrepo import fetch_ucirepo
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')

diabetes_data = fetch_ucirepo(id=296)

X = diabetes_data.data.features
y = diabetes_data.data.targets
y = y.values.ravel()

seed = 1234
np.random.seed(seed)  

#Process data
obj_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['number']).columns

num_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

obj_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transform, num_cols),
        ('cat', obj_transform, obj_cols)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

In [23]:
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [25]:
logistic_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('classifier', LogisticRegression(random_state=seed, max_iter=2000))])
logistic_pipeline.fit(X_train, y_train)
y_pred_logistic = logistic_pipeline.predict(X_test)

print("Logistic Regression")
print(classification_report(y_test, y_pred_logistic))

Logistic Regression
              precision    recall  f1-score   support

         <30       0.29      0.02      0.04      2852
         >30       0.50      0.33      0.40      8884
          NO       0.60      0.85      0.71     13706

    accuracy                           0.58     25442
   macro avg       0.46      0.40      0.38     25442
weighted avg       0.53      0.58      0.52     25442



In [26]:
random_forest_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('classifier', RandomForestClassifier(random_state=seed))])
random_forest_pipeline.fit(X_train, y_train)
y_pred_rf = random_forest_pipeline.predict(X_test)

print("Random Forest")
print(classification_report(y_test, y_pred_rf))

Random Forest
              precision    recall  f1-score   support

         <30       0.53      0.01      0.02      2852
         >30       0.51      0.32      0.40      8884
          NO       0.60      0.87      0.71     13706

    accuracy                           0.58     25442
   macro avg       0.55      0.40      0.38     25442
weighted avg       0.56      0.58      0.53     25442



In [36]:
knn_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('classifier', KNeighborsClassifier(n_neighbors=100))])
knn_pipeline.fit(X_train, y_train)
y_pred_knn = knn_pipeline.predict(X_test)

print("K-Nearest Neighbors")
print(classification_report(y_test, y_pred_knn))

K-Nearest Neighbors
              precision    recall  f1-score   support

         <30       0.41      0.01      0.02      2852
         >30       0.51      0.26      0.34      8884
          NO       0.59      0.90      0.71     13706

    accuracy                           0.58     25442
   macro avg       0.50      0.39      0.36     25442
weighted avg       0.54      0.58      0.51     25442



In [32]:
best_hyperparams = {
    'hidden_layer_sizes': [500],
    'solver': 'sgd',
    'learning_rate_init': 0.01,
    'max_iter': 20
}
nn_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('classifier', MLPClassifier(**best_hyperparams, shuffle=True, random_state=seed, verbose=False))])
nn_pipeline.fit(X_train, y_train)
y_pred_nn = nn_pipeline.predict(X_test)

print("Neural Networks")
print(classification_report(y_test, y_pred_nn))

Neural Networks
              precision    recall  f1-score   support

         <30       0.37      0.03      0.05      2852
         >30       0.51      0.33      0.40      8884
          NO       0.61      0.87      0.71     13706

    accuracy                           0.58     25442
   macro avg       0.49      0.41      0.39     25442
weighted avg       0.55      0.58      0.53     25442

